In [2]:
import rospy
import numpy as np
from panda_robot_custom import PandaRobotCustom
import my_log
import tools

log = my_log.logger()
log._print_logger_info()

rospy.init_node("franka_test", anonymous=True)

robot = PandaRobotCustom()
rospy.sleep(1)

2023-08-15 16:40:38 [INFO] Logger 'my_log' with level 20
2023-08-15 16:40:39 [INFO] Initializing PandaRobot class
2023-08-15 16:40:39 [INFO] Panda initialized


In [9]:
# EE frame transformation
T = np.array([[1.0, 0.0, 0.0, 0.2],
              [0.0, 1.0, 0.0, 0.0],
              [0.0, 0.0, 1.0, 0.1],
              [0.0, 0.0, 0.0, 1.0]])
rot = tools.get_rotation_matrix(angle=np.deg2rad(-45), axis="z")
T[0:3, 0:3] = rot

# EE frame parameters
mass = 0.76
F_x_center_load = [0.2, 0.1, 0.15]
load_inertia = [0.001333, 0.0,     0.0,
                0.0,      0.00347, 0.0,
                0.0,      0.0,     0.00453]

In [10]:
log.debug(robot.get_state())
robot.switch_controller(stop_controllers=robot.init_controllers)
rospy.sleep(1)

response = robot.set_EE_load(mass, F_x_center_load, load_inertia)
new_mtx = np.reshape(T, (16), order = 'F')
response = robot.set_NE_T_EE_transform(NE_T_EE=new_mtx)

robot.switch_controller(start_controllers=robot.init_controllers)
rospy.sleep(1)
log.debug(robot.get_state())